In [1]:
from scipy import spatial

def cosine(x, y):
    dataSetI = x
    dataSetII = y
    return 1 - spatial.distance.cosine(dataSetI, dataSetII)

In [2]:
import pickle

def load_category(category):
    pickle_in = open("elmo_category/reduced/"+ category +".pickle", "rb")
    return pickle.load(pickle_in)

def load_data(id_file):
    pickle_in = open("elmo_restaurant/"+ str(id_file) +".pickle", "rb")
    return pickle.load(pickle_in)[0]

In [3]:
import numpy as np

def similarity(category, word):
    res = []
    for cat in category:
        res.append(cosine(cat, word))
    return np.mean(res)

In [4]:
def sim_result(word):
    c1 = similarity(ambience, word)
    c2 = similarity(food, word)
    c3 = similarity(service, word)
    c4 = similarity(value, word)
    return c1, c2, c3, c4

In [5]:
def cat_str(a, b, c, d):
    if a > b and a > c and a > d:
        return 1
    elif b > a and b > c and b > d:
        return 2
    elif c > b and c > a and c > d:
        return 3
    elif d > b and d > c and d > a:
        return 4
    else:
        return 5

In [6]:
def str_cat(num):
    pred =''
    if num == 1:
        pred = 'AMBIENCE'
    elif num == 2:
        pred = 'FOOD'
    elif num == 3:
        pred = 'SERVICE'
    elif num == 4:
        pred = 'VALUE'
    return pred

In [7]:
ambience = load_category('ambience')
food = load_category('food')
service = load_category('service')
value = load_category('value')

In [15]:
import pandas as pd

df = pd.read_csv('lda.csv')
df[160:168]

,Unnamed: 0,id,review,category,term
160,160,161,we took advantage of the half price sushi deal...,FOOD,price|sushi
161,161,162,the food was well prepared and the service was...,FOOD|SERVICE,service
162,162,163,the prix fixe menu is worth every penny and yo...,FOOD|FOOD|FOOD,quality
163,163,164,"$6 and there is much tasty food, all of it fre...",FOOD|FOOD|FOOD,tasty|fresh|continually
164,164,165,"i am not a vegetarian but, almost all the dish...",FOOD,dish|almost
165,165,166,the food here is not good.,FOOD,NaN
166,166,167,"i like the somosas, chai, and the chole, but t...",FOOD|FOOD|FOOD|FOOD|FOOD,NaN
167,167,168,the service varys from day to day- sometimes t...,SERVICE,service|sometimes|times


In [25]:
import pandas as pd

sf = pd.DataFrame(columns=['id', 'review', 'label', 'pred_single', 'pred_multiple', 'terms'])
for index in range(0, len(df)):
    pred= ''
    pred_mult=[]
    if type(df['term'][index]) != float:
        terms = df['term'][index].split('|')
        id_file = df['id'][index]
        label = df['category'][index]
        review = df['review'][index]
        print('-', index, id_file)
        
        vector = load_data(id_file)
        tokenized = df['review'][index].split(' ')
        
        c1 = []
        c2 = []
        c3 = []
        c4 = []
        for term in terms:
            i = 0
            for token in tokenized:
                if term in token:
                    a, b, c, d = sim_result(vector[i])
                    x = cat_str(a, b, c, d)
                    pred_mult.append(str_cat(x))
                    
                    c1.append(a)
                    c2.append(b)
                    c3.append(c)
                    c4.append(d)
                i += 1
        am = np.mean(c1)
        fo = np.mean(c2)
        se = np.mean(c3)
        va = np.mean(c4)

        num = cat_str(am, fo, se, va)
        pred = str_cat(num)
    else:
        terms = ''
        id_file = df['id'][index]
        label = df['category'][index]
        review = df['review'][index]
#     print(review)
#     print(terms)
#     print(label)
#     print(pred)
#     print(pred_mult)
#     print(index, id_file, pred)
#     print()
    sf = sf.append({'id': id_file,
                    'review': review,
                    'label': label,
                    'pred_single': pred,
                    'pred_multiple': '|'.join(pred_mult),
                    'terms': '|'.join(terms)
                   }, ignore_index=True)

sf.to_csv("hasil_kategori_lda.csv")
sf.to_excel("hasil_kategori_lda.xlsx")

- 0 0
- 1 2
- 2 3
- 3 4
- 4 5
- 5 6
- 6 7
- 7 8
- 8 9
- 9 10
- 10 11
- 11 12
- 12 13
- 13 14
- 14 15
- 15 16
- 16 17
- 17 18
- 18 19
- 19 20
- 20 21
- 21 22
- 22 23
- 23 24
- 24 25
- 25 26
- 26 27
- 27 28
- 28 29
- 29 30
- 30 31
- 31 32
- 32 33
- 33 34
- 34 35
- 35 36
- 36 37
- 37 38
- 38 39
- 39 40
- 40 41
- 41 42
- 42 43
- 43 44
- 44 45
- 45 46
- 46 47
- 47 48
- 48 49
- 49 50
- 50 51
- 51 52
- 52 53
- 53 54
- 54 55
- 55 56
- 56 57
- 57 58
- 58 59
- 59 60
- 60 61
- 61 62
- 62 63
- 63 64
- 64 65
- 65 66
- 66 67
- 67 68
- 68 69
- 69 70
- 70 71
- 71 72
- 72 73
- 73 74
- 74 75
- 75 76
- 76 77
- 77 78
- 78 79
- 79 80
- 80 81
- 81 82
- 82 83
- 83 84
- 84 85
- 85 86
- 86 87
- 87 88
- 88 89
- 89 90
- 90 91
- 91 92
- 92 93
- 93 94
- 94 95
- 95 96
- 96 97
- 97 98
- 98 99
- 99 100
- 100 101
- 101 102
- 102 103
- 103 104
- 104 105
- 105 106
- 106 107
- 107 108
- 108 109
- 109 110
- 110 111
- 111 112
- 112 113
- 113 114
- 114 115
- 115 116
- 116 117
- 117 118
- 118 119
- 119 120
- 120 121
- 121 12